In [1]:
import folium
from src.apikeys import MONGO
from src import process_tweets
from pymongo import MongoClient
import pandas as pd

In [2]:
tweet_processor = process_tweets.TweetProcessor('../models/tfidf_logistic_reg.pk')


In [3]:
client = MongoClient('mongodb://{}:{}@localhost:27017'.format(MONGO.USER, MONGO.PASSWORD))

In [4]:
database = client['twitter_database']

In [5]:
database.collection_names()

['test',
 'trends_2017_10_16_17',
 'broncos_raiders',
 'available_trends',
 'test_method',
 'patriots_panthers']

In [6]:
table = database['trends_2017_10_16_17']

In [10]:
df = tweet_processor.process_database(table, topics=['trump'])

In [11]:
df.head(10)

,state,date,sentiment,time_bin
0,Missouri,2017-10-17 00:53:06+00:00,0.419920,0
1,None,2017-10-17 00:53:02+00:00,0.069031,0
2,Texas,2017-10-17 00:53:06+00:00,0.491284,0
3,None,2017-10-17 00:53:06+00:00,0.084560,0
4,New York,2017-10-17 00:53:07+00:00,0.160232,0
5,None,2017-10-17 00:53:08+00:00,0.491284,0
6,Texas,2017-10-17 00:53:08+00:00,0.491284,0
7,Oregon,2017-10-17 00:53:09+00:00,0.491284,0
8,None,2017-10-17 00:53:10+00:00,0.491284,0
9,None,2017-10-17 00:53:12+00:00,0.996493,0


In [12]:
from src.visualization import visualize_sentiment
my_map = visualize_sentiment(df[(df.sentiment>.6)|(df.sentiment<.4)])
my_map

In [16]:
my_map.save('../maps/test_map.html')

In [13]:
import numpy as np
a = df.groupby('state')['sentiment'].agg(['mean','std','count'])
a['mean_dev'] = a.apply(lambda row: row['std']/np.sqrt(row['count']),axis=1)
a[['mean','mean_dev']].head(10)

,mean,mean_dev
state,,
Alabama,0.474376,0.074061
Alaska,0.346408,0.131172
Arizona,0.467244,0.016641
Arkansas,0.084560,NaN
California,0.414040,0.020002
Colorado,0.355284,0.048769
Connecticut,0.332462,0.058009
Delaware,0.432106,0.063030
District of Columbia,0.331921,0.079300


In [14]:
a

,mean,std,count,mean_dev
state,,,,
Alabama,0.474376,0.305360,17,0.074061
Alaska,0.346408,0.227196,3,0.131172
Arizona,0.467244,0.068613,17,0.016641
Arkansas,0.084560,NaN,1,NaN
California,0.414040,0.244970,150,0.020002
Colorado,0.355284,0.182476,14,0.048769
Connecticut,0.332462,0.209156,13,0.058009
Delaware,0.432106,0.227258,13,0.063030
District of Columbia,0.331921,0.194245,6,0.079300
